# Batch Mode Demo Script

This is a Jupyter Notebook dumping ground for function testing before it goes into class objects and then to a working example in another notebook.

## This script
Tests functions for batch processing of images and calculating canopy openness

## Workflow
  - 1) Get input of a directory of images from user
  - 2) List of all jpg files within that directory
  - 3) Create empty dataframe with image information (plot, subplot, date, exposure)
  - 4) Iterate through folder, running ImagePrep, FishEye, and CanOpen modules
  - 5) Save openness metric as last column in the dataframe
  
 -----
 -----

## Setup

Loading packages, setting logger levels

In [1]:
#Importing my package
import CanopyOpenness
from CanopyOpenness import ImageLoad
from CanopyOpenness import FishEye
from CanopyOpenness import CanOpen
#Importing packages
import glob #helping identify files in pathfiles
import os #finding pathfiles
import pathlib #getting pathfiles
import pandas as pd #pandas for dataframe manipulation
import numpy as np #statistical calculations
import natsort #batch loading of files
import skimage #image manipulation
from skimage import io #filepaths in skimage
import warnings #warnings package
from loguru import logger #Logger for debugging messages

In [2]:
#Setting logging level
CanopyOpenness.set_loglevel("DEBUG")

## Getting image files from directory

Getting user input (filepath for directory containing image files.  
Then saving each individual image as an item in a list.  
Then creating a dataframe with metadata from image names, and an empty column to hold canopy openness values

In [3]:
#Defining user input
filepath = './examples/Batch_Test/' #where photos are stored

In [4]:
#iterate through all images in the folder and save as a list
all_images = [f for f in os.listdir(filepath) if f.endswith('JPG')]

#sort the list
all_images.sort()

#check
all_images

['LFDP.394.12Dec2017.EV-2.AF.JPG',
 'LFDP.394.12Dec2017.EV-4.AF.JPG',
 'LFDP.394.12Dec2017.EV0.AF.JPG',
 'LFDP.397.12Dec2017.EV-2.AF.JPG',
 'LFDP.397.12Dec2017.EV-4.AF.JPG',
 'LFDP.397.12Dec2017.EV0.AF.JPG',
 'LFDP.400.11Dec2017.EV-2.M.JPG',
 'LFDP.400.11Dec2017.EV-4.M.JPG',
 'LFDP.400.11Dec2017.EV0.M.JPG',
 'LFDP.403.11Dec2017.EV-2.AF.JPG',
 'LFDP.403.11Dec2017.EV-4.AF.JPG',
 'LFDP.403.11Dec2017.EV0.AF.JPG',
 'LFDP.406.11Dec2017.EV-2.AF.JPG',
 'LFDP.406.11Dec2017.EV-4.AF.JPG',
 'LFDP.406.11Dec2017.EV0.AF.JPG']

In [5]:
# create an Empty DataFrame object
df = pd.DataFrame(columns=['Plot','Subplot','Date','Exposure','Openness'])

#split image by period to get different columns
df['Plot'] = [i.split('.')[0] for i in all_images]  # get first item in list of images
df['Subplot'] = [i.split('.')[1] for i in all_images]  # get subplot information (second item)
df['Date'] = [i.split('.')[2] for i in all_images]  # get date information (third item)
df['Exposure'] = [i.split('.')[3] for i in all_images]  # get exposure information (fourth item)

#check
df

,Plot,Subplot,Date,Exposure,Openness
0,LFDP,394,12Dec2017,EV-2,NaN
1,LFDP,394,12Dec2017,EV-4,NaN
2,LFDP,394,12Dec2017,EV0,NaN
3,LFDP,397,12Dec2017,EV-2,NaN
4,LFDP,397,12Dec2017,EV-4,NaN
5,LFDP,397,12Dec2017,EV0,NaN
6,LFDP,400,11Dec2017,EV-2,NaN
7,LFDP,400,11Dec2017,EV-4,NaN
8,LFDP,400,11Dec2017,EV0,NaN
9,LFDP,403,11Dec2017,EV-2,NaN


## Batch processing function

Code to create an empty list to hold openness values.  
Then iterating over all the image files and running ImagePrep, FishEye, and CanOpen modules to calculate openness for each image.  
Then storing those values in the empty column in the dataframe containing the image metadata.  
Finally storing the resultant dataframe as a csv for the user

In [6]:
#create empty list to hold results
results = []

#iterate through list of images and run CanopyOpenness functions
for image in all_images:
    
    #load image and threshold using isodata algorithm, don't plot, set to batch
    img = ImageLoad.ImagePrep(filepath,image,threshold_method="isodata",plot=False,batch=True)
    #load image
    og = img.imageLoad()
    #turn blue 
    blue = img.BluePic()
    #threshold algorithm and turn to black and white
    bw = img.bwPic()
    
    #set fisheye coordinates for center lens, don't plot, set to batch
    fish = FishEye.FishEye(bw,plot=False,batch=True)
    #save image array with coordinates
    fishy = fish.CircleCoords()
    
    #run canopy openness module, set to batch
    gfp = CanOpen.CanOpen(fishy,batch=True) #running module
    gaps=gfp.calc_gap_fractions() #calculating array of proportion sky for 89 sub-circles within fisheye lens
    openness=gfp.openness() #openness calculation
    
    #print message to user
    print("Image",image, "processed")

    #appending to result list
    results.append(openness)

Image LFDP.394.12Dec2017.EV-2.AF.JPG processed
Image LFDP.394.12Dec2017.EV-4.AF.JPG processed
Image LFDP.394.12Dec2017.EV0.AF.JPG processed
Image LFDP.397.12Dec2017.EV-2.AF.JPG processed
Image LFDP.397.12Dec2017.EV-4.AF.JPG processed
Image LFDP.397.12Dec2017.EV0.AF.JPG processed
Image LFDP.400.11Dec2017.EV-2.M.JPG processed
Image LFDP.400.11Dec2017.EV-4.M.JPG processed
Image LFDP.400.11Dec2017.EV0.M.JPG processed
Image LFDP.403.11Dec2017.EV-2.AF.JPG processed
Image LFDP.403.11Dec2017.EV-4.AF.JPG processed
Image LFDP.403.11Dec2017.EV0.AF.JPG processed
Image LFDP.406.11Dec2017.EV-2.AF.JPG processed
Image LFDP.406.11Dec2017.EV-4.AF.JPG processed
Image LFDP.406.11Dec2017.EV0.AF.JPG processed


In [7]:
#add values from result to the dataframe
df['Openness'] = results

#check
df

,Plot,Subplot,Date,Exposure,Openness
0,LFDP,394,12Dec2017,EV-2,0.382384
1,LFDP,394,12Dec2017,EV-4,0.333713
2,LFDP,394,12Dec2017,EV0,0.515455
3,LFDP,397,12Dec2017,EV-2,0.616088
4,LFDP,397,12Dec2017,EV-4,0.596442
5,LFDP,397,12Dec2017,EV0,0.693544
6,LFDP,400,11Dec2017,EV-2,0.375623
7,LFDP,400,11Dec2017,EV-4,0.303961
8,LFDP,400,11Dec2017,EV0,0.563061
9,LFDP,403,11Dec2017,EV-2,0.588104


In [8]:
#saving dataframe as a csv
df.to_csv(r'./Batch_Test_Dataframe.csv', index = False)